# Setup

## Libraries + dataset 

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf
import openpyxl

username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
            complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

    elif(username == "davidguzman"):
        pathPart1 = "/Users/"
        pathPart2 = "/documents/Github/SCGProject/Datasets/"
        complePath = pathPart1+username+pathPart2

        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
            complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

elif platform.system() == "Linux":
    complePath = "/home/alinux/unibg/git_repo/SCGProject"
    pathPart1 = "/Users/"
    pathPart2 = "/documents/Github/SCGProject/Datasets/"
    complePath = pathPart1+username+pathPart2
    complePath = '/home/alinux/unibg/git_repo/SCGProject/Datasets/'

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")

else:
    complePath = "C:/SCGProject/Datasets/"

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")


dfCambio["TassoCambioMedio"] = dfCambio["TassoCambioMedio"].str.replace(
    ',', '.')


Username:  davidguzman


## Removing the intermediate in the dataset

### Budget

In [2]:
dfConsumi.rename(columns={'Budget/cons': 'budget'}, inplace=True)
qtaArtBudget = sqldf(
    'select NrArticolo, count(*) as qta from dfConsumi where budget = "BUDGET" group by NrArticolo')

dfVendite.rename(columns={'budget/cons': 'budget'}, inplace=True)
qtaArtVendBudget = sqldf(
    'select NrArticolo, count(*) as qta from dfVendite where budget = "BUDGET" group by NrArticolo')

artProdeVendBudget = sqldf(
    'select P.NrArticolo from qtaArtBudget P join qtaArtVendBudget V on P.NrArticolo = V.NrArticolo')


consumiB = sqldf('select * from dfConsumi where budget = "BUDGET"')


# ho trovato articoli che hanno consumato risorse a budget ma non sono stati messi tra le vendite a budget
artProdeNONVendBudget = sqldf(
    'select * from consumiB group by NrArticolo having NrArticolo not in (select NrArticolo from artProdeVendBudget)')

# prendo gli articoli che usano come materia prima almeno un articolo prodotto internamente
# sqldf('select NrMovimento, budget, C.NrArticolo, CodiceMP, NrDocumento, QuantityMPImpiegata, ImportoCostoTOTALE from consumiB C join artProdeNONVendBudget A on C.CodiceMP = A.NrArticolo')

costoPerArticoloBudget = sqldf(
    'select NrArticolo, sum(ImportoCostoTOTALE) as costoMpPerArt from consumiB where NrArticolo not in (select NrArticolo from artProdeNONVendBudget) group by NrArticolo')
sqldf('select sum(costoMpPerArt) as costoTotBudget from costoPerArticoloBudget')

mpB = sqldf('''select NrMovimento,budget,CodiceMP,NrArticolo, NrDocumento, sum(QuantityMPImpiegata), sum(ImportoCostoTOTALE) from consumiB group by NrArticolo''')
prodInternaB = sqldf('''select A.NrMovimento, A.budget, A.CodiceMP, A.NrArticolo, A.NrDocumento, A.QuantityMPImpiegata, A.ImportoCostoTOTALE
         from consumiB A join mpB B on A.CodiceMP = B.NrArticolo''')

mpIntermediB = sqldf('''select distinct CodiceMP from prodInternaB''')
consumiBNOArtInterm = sqldf(
    '''select * from consumiB where NrArticolo not in (select CodiceMP from mpIntermediB)''')


### Consuntivo

In [3]:
consumiC = sqldf(
    '''select * from dfConsumi where budget = "CONSUNTIVO" or budget = "Consuntivo"''')
mpC = sqldf('''select NrMovimento,budget,CodiceMP,NrArticolo, NrDocumento, sum(QuantityMPImpiegata), sum(ImportoCostoTOTALE) from consumiC group by NrArticolo''')
#A.NrArticolo, A.CodiceMP, A.NrDocumento, A.QuantityMPImpiegata, A.ImportoCostoTOTALE, A.CodiceMP
prodInternaC = sqldf('''select A.NrMovimento, A.budget, A.CodiceMP, A.NrArticolo, A.NrDocumento, A.QuantityMPImpiegata, A.ImportoCostoTOTALE
         from consumiC A join mpC B on A.CodiceMP = B.NrArticolo''')
mpIntermediC = sqldf('''select distinct CodiceMP from prodInternaC''')
consumiCNOArtInterm = sqldf(
    '''select * from consumiC where NrArticolo NOT IN (select CodiceMP from mpIntermediC)''')


### UNIONE DELLE DUE TABELLE PER FARE IL NUOVO DATASET

In [4]:
dfConsumi = sqldf('''  SELECT *
                            FROM consumiBNOArtInterm
                            UNION
                            SELECT *
                            FROM consumiCNOArtInterm
                            ''')
del dfConsumi['Unnamed: 0']
dfConsumi


,NrMovimento,budget,CodiceMP,NrArticolo,NrDocumento,QuantityMPImpiegata,ImportoCostoTOTALE
0,28854,BUDGET,ART0001074,ART0000018,ODP17-05342,30.0,5.15
1,66819,CONSUNTIVO,ART0001074,ART0000018,ODP19-05998,30.0,5.15
2,28853,BUDGET,ART0005696,ART0000041,ODP17-05342,4.0,92.00
3,66818,CONSUNTIVO,ART0005696,ART0000041,ODP19-05998,4.0,88.00
4,28855,BUDGET,ART0005697,ART0000042,ODP17-05342,4.0,104.00
...,...,...,...,...,...,...,...
4046,66172,CONSUNTIVO,ART0005850,ART0005136,ODP19-02057,155.0,1.23
4047,66235,CONSUNTIVO,ART0006245,ART0005136,ODP19-02528,700.0,0.00
4048,66446,CONSUNTIVO,ART0006381,ART0005136,ODP19-02674,300.0,1.62
4049,66594,CONSUNTIVO,ART0006345,ART0005136,ODP19-06688,9.0,0.00


# Funzione che fa partire l'analisi degli scostamenti

## Reading delle tabelle che servono 

In [5]:
venditeS1   = pd.read_csv("venditeS1.csv")
mergeImpiego = pd.read_csv("mergeImpieg.csv")
ConsumiUnit = pd.read_csv("ConsumiUnit.csv")


## By articolo full stack!

In [6]:
def by_articolo_full(venditeS1, mergeImpiego, ConsumiUnit, articolo):

        venditeSingoloArticolo = sqldf('select * from venditeS1 where NrArticolo = "'+articolo+'"')

        BudgetStandardRic = sqldf('select sum(Mix_y*VolumeTotale_y*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeSingoloArticolo ').iloc[0][0]
        MixStandardRic = sqldf('select sum(Mix_y*VolumeTotale_x*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeSingoloArticolo ').iloc[0][0]
        MixEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeSingoloArticolo ').iloc[0][0]
        BudgetEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x/TassoCambioMedio_y)/100000000000 from venditeSingoloArticolo ').iloc[0][0]
        BudgetEffettivoValutaRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x/TassoCambioMedio_x)/100000000000 from venditeSingoloArticolo ').iloc[0][0]

        if(BudgetStandardRic is None):
                BudgetStandardRic = 0

        if(MixStandardRic is None):
                MixStandardRic = 0

        if(MixEffettivoRic is None):
                MixEffettivoRic = 0

        if(BudgetEffettivoRic is None):
                BudgetEffettivoRic = 0

        if(BudgetEffettivoValutaRic is None):
                BudgetEffettivoValutaRic = 0

        ScostamentoVolumeRic = MixStandardRic-BudgetStandardRic
        ScostamentoMixRic = MixEffettivoRic-MixStandardRic
        ScostamentoPrezzoRic = BudgetEffettivoRic-MixEffettivoRic
        ScostamentoValutaRic = BudgetEffettivoValutaRic-BudgetEffettivoRic


        lavorazioniSingoloProdotto = sqldf('select * from mergeImpiego where NrArticolo = "'+articolo+'"')

        #Budget Standard
        BudgetStandardLav = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*QuantitydiOutput_y) from lavorazioniSingoloProdotto').iloc[0][0]
        # Impiego Standard (volume a consuntivo)
        ImpiegoStandardLav = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*QuantitydiOutput_x) from lavorazioniSingoloProdotto').iloc[0][0]
        # Impiego Effettivo (volume e impiego a consuntivo)
        ImpiegoEffettivoLav=sqldf('select sum(ImpiegoUnitario_x*CostoOrario_y*QuantitydiOutput_x) from lavorazioniSingoloProdotto').iloc[0][0]
        # Budget Effettivo (tutto a consuntivo)
        BudgetEffettivoLav=sqldf('select sum(ImpiegoUnitario_x*CostoOrario_x*QuantitydiOutput_x) from lavorazioniSingoloProdotto').iloc[0][0]

        if(BudgetStandardLav is None):
                        BudgetStandardLav = 0

        if(ImpiegoStandardLav is None):
                        ImpiegoStandardLav = 0

        if(ImpiegoEffettivoLav is None):
                        ImpiegoEffettivoLav = 0

        if(BudgetEffettivoLav is None):
                        BudgetEffettivoLav = 0

        #Scostamenti
        ScostamentoVolumeLav = ImpiegoStandardLav-BudgetStandardLav
        ScostamentoImpiegoLav = ImpiegoEffettivoLav-ImpiegoStandardLav
        ScostamentoPrezzoLav = BudgetEffettivoLav-ImpiegoEffettivoLav

        consumiSingoloArticolo = sqldf('select * from ConsumiUnit where NrArticolo="'+articolo+'"')

        BudgetEffettivoMP = sqldf('select sum(QuantitydiOutput_x*ImpiegoPerPezzo_x*CostoPerUnitaMis_x) from consumiSingoloArticolo').iloc[0][0]
        ImpiegoEffettivoMP = sqldf('select sum(QuantitydiOutput_x*ImpiegoPerPezzo_x*CostoPerUnitaMis_y) from consumiSingoloArticolo').iloc[0][0]
        ImpiegoStandardMP = sqldf('select sum(QuantitydiOutput_x*ImpiegoPerPezzo_y*CostoPerUnitaMis_y) from consumiSingoloArticolo').iloc[0][0]
        BudgetStandardMP = sqldf('select sum(QuantitydiOutput_y*ImpiegoPerPezzo_y*CostoPerUnitaMis_y) from consumiSingoloArticolo').iloc[0][0]

        #TODO: choose strategy 

        BudgetEffettivoMP = sqldf('select (avg(QuantitydiOutput_x)*avg(ImpiegoPerPezzo_x)*avg(CostoPerUnitaMis_x)) from consumiSingoloArticolo group by NrArticolo').iloc[0][0]
        ImpiegoEffettivoMP = sqldf('select (avg(QuantitydiOutput_x)*avg(ImpiegoPerPezzo_x)*avg(CostoPerUnitaMis_y)) from consumiSingoloArticolo group by NrArticolo').iloc[0][0]
        ImpiegoStandardMP = sqldf('select (avg(QuantitydiOutput_x)*avg(ImpiegoPerPezzo_y)*avg(CostoPerUnitaMis_y)) from consumiSingoloArticolo group by NrArticolo').iloc[0][0]
        BudgetStandardMP = sqldf('select (avg(QuantitydiOutput_y)*avg(ImpiegoPerPezzo_y)*avg(CostoPerUnitaMis_y)) from consumiSingoloArticolo group by NrArticolo').iloc[0][0]

        if(BudgetStandardMP is None):
                BudgetStandardMP = 0

        if(ImpiegoStandardMP is None):
                ImpiegoStandardMP = 0

        if(ImpiegoEffettivoMP is None):
                ImpiegoEffettivoMP = 0

        if(BudgetEffettivoMP is None):
                BudgetEffettivoMP = 0

        ScostamentoVolumeMP =  ImpiegoStandardMP - BudgetStandardMP
        ScostamentoImpiegoMP = ImpiegoEffettivoMP-ImpiegoStandardMP
        ScostamentoPrezzoMP = BudgetEffettivoMP-ImpiegoEffettivoMP 

        data = {articolo: ['Ricavi', 'CostiMP', 'CostiLav', 'Margine'], 
                'Budget' : [BudgetStandardRic, BudgetStandardMP,BudgetStandardLav, (BudgetStandardRic - (BudgetStandardMP+BudgetStandardLav))],
                'ScostamentoVolume' : [ScostamentoVolumeRic,ScostamentoVolumeMP,ScostamentoVolumeLav, (ScostamentoVolumeRic - (ScostamentoVolumeMP+ScostamentoVolumeLav))], 
                'MixStandard' : [MixStandardRic, ImpiegoStandardMP,ImpiegoStandardLav, (MixStandardRic - (ImpiegoStandardMP+ImpiegoStandardLav))],
                'ScostamentoMix' : [ScostamentoMixRic,ScostamentoImpiegoMP,ScostamentoImpiegoLav, (ScostamentoMixRic - (ScostamentoImpiegoMP+ScostamentoImpiegoLav))],
                'MixEffettivo' : [MixEffettivoRic, ImpiegoEffettivoMP,ImpiegoEffettivoLav, (MixEffettivoRic - (ImpiegoEffettivoMP+ImpiegoEffettivoLav))],
                'ScostamentoPrezzo' : [ScostamentoPrezzoRic,ScostamentoPrezzoMP,ScostamentoPrezzoLav, (ScostamentoPrezzoRic - (ScostamentoPrezzoMP+ScostamentoPrezzoLav))], 
                'MixValuta' : [BudgetEffettivoRic, BudgetEffettivoMP,BudgetEffettivoLav, (BudgetEffettivoRic - (BudgetEffettivoMP+BudgetEffettivoLav))],
                'ScostamentoValuta' : [ScostamentoValutaRic, 0, 0, (ScostamentoValutaRic - (0+0))],
                'Consuntivo' : [BudgetEffettivoValutaRic, BudgetEffettivoMP, BudgetEffettivoLav, (BudgetEffettivoValutaRic - (BudgetEffettivoMP+BudgetEffettivoLav))],
                'ScostamentoTotale' : [BudgetEffettivoValutaRic-BudgetStandardRic,BudgetEffettivoMP-BudgetStandardMP,BudgetEffettivoLav-BudgetStandardLav, (BudgetEffettivoValutaRic - BudgetStandardRic - (BudgetEffettivoMP-BudgetStandardMP+BudgetEffettivoLav-BudgetStandardLav))]}

        return pd.DataFrame(data)


ART0001391 da errore perché viene rimosso dal filtro degli articoli intermedi

In [7]:
articoli = sqldf('select distinct NrArticolo from dfConsumi')
scos = []

for articolo in articoli.values:
   theArticolo = articolo[0]
   print(theArticolo)
   scos.append(by_articolo_full(
       venditeS1, mergeImpiego, ConsumiUnit, theArticolo))


ART0000018
ART0000041
ART0000042
ART0000043
ART0000115
ART0000116
ART0000119
ART0000121
ART0000125
ART0000128
ART0000129
ART0000131
ART0000151
ART0000152
ART0000157
ART0000223
ART0000226
ART0000227
ART0000228
ART0000229
ART0000308
ART0000312
ART0000313
ART0000320
ART0000323
ART0000324
ART0000325
ART0000456
ART0000555
ART0000625
ART0000627
ART0000628
ART0000630
ART0000635
ART0000638
ART0000639
ART0000641
ART0000642
ART0000643
ART0000644
ART0000645
ART0000690
ART0000697
ART0000748
ART0000749
ART0000752
ART0000753
ART0000756
ART0000757
ART0000758
ART0000762
ART0000765
ART0000769
ART0000772
ART0000778
ART0000779
ART0000780
ART0000782
ART0000783
ART0000784
ART0000786
ART0000788
ART0000790
ART0000795
ART0000796
ART0000797
ART0000804
ART0000805
ART0000811
ART0000812
ART0000814
ART0000825
ART0000826
ART0000830
ART0000841
ART0000843
ART0000845
ART0000847
ART0000848
ART0000849
ART0000850
ART0000851
ART0000853
ART0000877
ART0000878
ART0000879
ART0000880
ART0000881
ART0000882
ART0000883
ART0000885

# Per ognuna delle variabili prendo il massimo

In [ ]:
maxSV = scos[0].iloc[3, 2]
indmaxSV = scos[0].columns[0]
maxSM = scos[0].iloc[3,4]
indmaxSM = scos[0].columns[0]
maxSP = scos[0].iloc[3,6] 
indmaxSP = scos[0].columns[0]
maxSVa = scos[0].iloc[3,8]
indmaxSVa = scos[0].columns[0]

for i in scos:
    if maxSV < i.iloc[3, 2]:
        maxSV = i.iloc[3, 2]
        indmaxSV = i.columns[0]
    if maxSM < i.iloc[3, 4]:
        maxSM = i.iloc[3, 4]
        indmaxSM = i.columns[0]
    if maxSP < i.iloc[3, 6]:
        maxSP = i.iloc[3, 6]
        indmaxSP = i.columns[0]
    if maxSVa < i.iloc[3, 8]:
        maxSVa = i.iloc[3, 8]
        indmaxSVa = i.columns[0]


## Scrittura dei risultati per il massimo

In [ ]:
print("Massimo scostamento di volume: "+str(maxSV)+" su articolo "+str(indmaxSV))
print("Massimo scostamento di mix: " + str(maxSM)+" su articolo "+str(indmaxSM))
print("Massimo scostamento di prezzo: "+str(maxSP)+" su articolo "+str(indmaxSP))
print("Massimo scostamento di valuta: "+str(maxSVa)+" su articolo "+str(indmaxSVa))


Massimo scostamento di volume: 11484.751942702851 su articolo ART0000762
Massimo scostamento di mix: 13152.407764994563 su articolo ART0000847
Massimo scostamento di prezzo: 25480.076379784445 su articolo ART0002370
Massimo scostamento di valuta: 0.0 su articolo ART0000115


# Per ognuna delle variabili prendo il minimo

In [ ]:
minSV = scos[0].iloc[3, 2]
indminSV = scos[0].columns[0]
minSM = scos[0].iloc[3, 4]
indminSM = scos[0].columns[0]
minSP = scos[0].iloc[3, 6]
indminSP = scos[0].columns[0]
minSVa = scos[0].iloc[3, 8]
indminSVa = scos[0].columns[0]

for i in scos:
    if minSV > i.iloc[3, 2]:
        minSV = i.iloc[3, 2]
        indmindminSVinV = i.columns[0]
    if minSM > i.iloc[3, 4]:
        minSM = i.iloc[3, 4]
        indminSM = i.columns[0]
    if minSP > i.iloc[3, 6]:
        minSP = i.iloc[3, 6]
        indminSP = i.columns[0]
    if minSVa > i.iloc[3, 8]:
        minSVa = i.iloc[3, 8]
        indminSVa = i.columns[0]


## Scrittura dei risultati per il minimo

In [ ]:
print("Minimo scostamento di volume: " +
      str(minSV)+" su articolo "+str(indmindminSVinV))
print("Minimo scostamento di mix: " + str(minSM)+" su articolo "+str(indminSM))
print("Minimo scostamento di prezzo: " +
      str(minSP)+" su articolo "+str(indminSP))
print("Minimo scostamento di valuta: " +
      str(minSVa)+" su articolo "+str(indminSVa))


Minimo scostamento di volume: -6031.387576120482 su articolo ART0001174
Minimo scostamento di mix: -31411.37854232556 su articolo ART0000762
Minimo scostamento di prezzo: -22293.101512303983 su articolo ART0005136
Minimo scostamento di valuta: -6104.642532125261 su articolo ART0000748


## Range di scostamento

In [ ]:
print("Range di scostamento su volume: ["+str(minSV)+" , "+str(maxSV)+"]")
print("Range di scostamento su mix: ["+str(minSM)+" , "+str(maxSM)+"]")
print("Range di scostamento su prezzo: ["+str(minSP)+" , "+str(maxSP)+"]")
print("Range di scostamento su valuta:["+str(minSVa)+" , "+str(maxSVa)+"]")

Range di scostamento su volume: [-6031.387576120482 , 11484.751942702851]
Range di scostamento su mix: [-31411.37854232556 , 13152.407764994563]
Range di scostamento su prezzo: [-22293.101512303983 , 25480.076379784445]
Range di scostamento su valuta:[-6104.642532125261 , 0.0]


# Sort degli scostamenti

In [8]:
scos[0]

,ART0000018,Budget,ScostamentoVolume,MixStandard,ScostamentoMix,MixEffettivo,ScostamentoPrezzo,MixValuta,ScostamentoValuta,Consuntivo,ScostamentoTotale
0,Ricavi,85.380891,24.879122,110.260013,-24.879119,85.380894,18.973532,104.354425,-12.634255,91.72017,6.339279
1,CostiMP,5.150000,0.000000,5.150000,0.000000,5.150000,0.000000,5.150000,0.000000,5.15000,0.000000
2,CostiLav,106.250000,0.000000,106.250000,12.750000,119.000000,11.000000,130.000000,0.000000,130.00000,23.750000
3,Margine,-26.019109,24.879122,-1.139987,-37.629119,-38.769106,7.973532,-30.795575,-12.634255,-43.42983,-17.410721


In [17]:
def Func(x):
  return x.iloc[3, 8]

scos.sort(key=Func)
scos


[  ART0000748        Budget  ScostamentoVolume   MixStandard  ScostamentoMix  \
 0     Ricavi  14995.171233        4369.440123  19364.611356    10224.522518   
 1    CostiMP   1720.289608        1032.173765   2752.463373     2331.477400   
 2   CostiLav    711.875000         427.125000   1139.000000     2069.750000   
 3    Margine  12563.006625        2910.141358  15473.147983     5823.295118   
 
    MixEffettivo  ScostamentoPrezzo     MixValuta  ScostamentoValuta  \
 0  29589.133874       20833.027206  50422.161080       -6104.642532   
 1   5083.940773        -436.384855   4647.555918           0.000000   
 2   3208.750000         566.250000   3775.000000           0.000000   
 3  21296.443101       20703.162061  41999.605161       -6104.642532   
 
      Consuntivo  ScostamentoTotale  
 0  44317.518548       29322.347315  
 1   4647.555918        2927.266310  
 2   3775.000000        3063.125000  
 3  35894.962629       23331.956004  ,
   ART0000853        Budget  ScostamentoVolum

In [26]:
for i in range(0,417):
    print(scos[i].iloc[3, 8])

-6104.642532125261
-4685.700782991349
-3799.465191223433
-3613.856491395454
-2540.6339160827556
-2449.896991954898
-2170.7948046093297
-2163.5588219272977
-1860.6812602212049
-1335.6475444549342
-1253.662713788568
-1094.605540321183
-982.715361030926
-832.7122901099774
-822.3751735602418
-675.3583827561761
-625.9699302744666
-588.0671639477387
-588.0671632162621
-585.7700240383992
-542.698698741458
-534.0993418226453
-508.58620805192504
-505.829644287277
-465.1703122426147
-441.0503724121968
-418.4465412696727
-392.81048792961246
-364.55569844695583
-342.8936921106347
-326.7448175620357
-310.1135431023258
-282.5478948265636
-275.65648275762305
-261.8736545480426
-253.83367787264478
-233.7337251662466
-229.71373563135262
-225.11946091872528
-215.93091149347129
-215.47148402220887
-204.44522280463457
-202.88317130961082
-199.85094689192738
-199.48340802226676
-188.13554948207752
-186.0681254481026
-179.17671379245485
-176.76471865219355
-176.76471865219355
-172.28530172351452
-172.285300

In [25]:
scos[1]

,ART0000853,Budget,ScostamentoVolume,MixStandard,ScostamentoMix,MixEffettivo,ScostamentoPrezzo,MixValuta,ScostamentoValuta,Consuntivo,ScostamentoTotale
0,Ricavi,32451.323398,9455.985016,41907.308413,-6871.366352,35035.942061,3666.268332,38702.210393,-4685.700783,34016.509610,1565.186212
1,CostiMP,4918.742222,1639.580741,6558.322963,696.821815,7255.144778,886.793000,8141.937778,0.000000,8141.937778,3223.195556
2,CostiLav,286.875000,95.625000,382.500000,116.875000,499.375000,88.125000,587.500000,0.000000,587.500000,300.625000
3,Margine,27245.706175,7720.779275,34966.485450,-7685.063167,27281.422283,2691.350332,29972.772615,-4685.700783,25287.071832,-1958.634343
